In [9]:
import os
import pandas as pd
import openai
import json
import requests

#OPENAI_API_KEY = "..."
#openai.my_api_key = OPENAI_API_KEY 

Below code makes a file called output file that has the csv files for the different scenarios converted to text files. I then manually took the samples i wanted out to put in testing and training data

In [3]:
script_directory = os.getcwd()

normal_VL_csv_files = []
folder_path = os.path.join(script_directory, "Normal")
scenario_path = os.path.join(folder_path, "Variable_Loading")
for root, dirs, files in os.walk(scenario_path):
    for file in files:
        if file.endswith('.csv'):
            normal_VL_csv_files.append(os.path.join(root, file))

normal_NVL_csv_files = []
folder_path = os.path.join(script_directory, "Normal")
scenario_path = os.path.join(folder_path, "No_Variable_Loading")
for root, dirs, files in os.walk(scenario_path):
    for file in files:
        if file.endswith('.csv'):
            normal_NVL_csv_files.append(os.path.join(root, file))

disturbance_UF_csv_files = []
folder_path = os.path.join(script_directory, "Disturbance")
scenario_path = os.path.join(folder_path, "Under frequency")
for root, dirs, files in os.walk(scenario_path):
    for file in files:
        if file.endswith('.csv'):
            disturbance_UF_csv_files.append(os.path.join(root, file))

disturbance_BP_csv_files = []
folder_path = os.path.join(script_directory, "Disturbance")
scenario_path = os.path.join(folder_path, "Busbar Protection")
for root, dirs, files in os.walk(scenario_path):
    for file in files:
        if file.endswith('.csv'):
            disturbance_BP_csv_files.append(os.path.join(root, file))

disturbance_BF_csv_files = []
folder_path = os.path.join(script_directory, "Disturbance")
scenario_path = os.path.join(folder_path, "Breaker Failure")
for root, dirs, files in os.walk(scenario_path):
    for file in files:
        if file.endswith('.csv'):
            disturbance_BF_csv_files.append(os.path.join(root, file))

attack_C_csv_files = []
folder_path = os.path.join(script_directory, "Attack")
attack_C_csv_files.append(os.path.join(folder_path,"CompositeAttack.csv"))

attack_DM_csv_files = []
folder_path = os.path.join(script_directory, "Attack")
scenario_path = os.path.join(folder_path, "Data Manipulation (DM)")
for root, dirs, files in os.walk(scenario_path):
    for file in files:
        if file.endswith('.csv'):
            attack_DM_csv_files.append(os.path.join(root, file))

attack_DM_csv_files = []
folder_path = os.path.join(script_directory, "Attack")
scenario_path = os.path.join(folder_path, "Data Manipulation (DM)")
for root, dirs, files in os.walk(scenario_path):
    for file in files:
        if file.endswith('.csv'):
            attack_DM_csv_files.append(os.path.join(root, file))

attack_DoS_csv_files = []
folder_path = os.path.join(script_directory, "Attack")
scenario_path = os.path.join(folder_path, "Denial of Service (DoS)")
for root, dirs, files in os.walk(scenario_path):
    for file in files:
        if file.endswith('.csv'):
            attack_DoS_csv_files.append(os.path.join(root, file)) 

attack_MS_csv_files = []
folder_path = os.path.join(script_directory, "Attack")
scenario_path = os.path.join(folder_path, "Message Suppression (MS)")
for root, dirs, files in os.walk(scenario_path):
    for file in files:
        if file.endswith('.csv'):
            attack_MS_csv_files.append(os.path.join(root, file)) 

scenarios = [normal_VL_csv_files, normal_NVL_csv_files, disturbance_UF_csv_files, disturbance_BP_csv_files, disturbance_BF_csv_files, attack_C_csv_files, attack_DM_csv_files, attack_DoS_csv_files, attack_MS_csv_files]

# Function to convert CSV data to text
def convert_to_txt(csv_file, output_txt):
    # Read CSV file
    data = pd.read_csv(csv_file)
    
    # Write data to text file
    with open(output_txt, 'w') as txt_file:
        # Write column names
        txt_file.write(','.join(data.columns) + '\n')
        # Write data rows
        for index, row in data.iterrows():
            txt_file.write(','.join(map(str, row.values)) + '\n')

# Define output directory
output_directory = "output_text_files"

# Create output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Convert each CSV file to text
for scenario_files in scenarios:
    scenario_category = os.path.basename(os.path.dirname(scenario_files[0]))
    for csv_file in scenario_files:
        # Define output directory based on the scenario category
        scenario_output_directory = os.path.join(output_directory, scenario_category)
        # Create scenario output directory if it doesn't exist
        if not os.path.exists(scenario_output_directory):
            os.makedirs(scenario_output_directory)
        # Define output text file path
        output_txt = os.path.join(scenario_output_directory, os.path.splitext(os.path.basename(csv_file))[0] + ".txt")
        convert_to_txt(csv_file, output_txt)

After manually seperating a test and train set, i made turn them into fine tuning prompts with context for the model, the text data as user input, and correct answer for sample response. Dont try to print in the notebook itself, look at jsonl file separatley 

In [11]:
def read_text_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()

folder_path = os.path.join(script_directory, "Training")

# Define the file paths and corresponding messages
file_messages = [
    ("BreakFailure.txt", "There is a Breaker Failure disturbance."),
    ("BusbarProtection.txt", "There is a Busbar Protection disturbance."),
    ("DM_AS1.txt", "There is a Data Manipulation attack."),
    ("DM_AS2.txt", "There is a Data Manipulation attack."),
    ("DoS_AS1.txt", "There is a Denial of Service attack."),
    ("MS_AS1.txt", "There is a Message Suppression attack."),
    ("MS_AS2.txt", "There is a Message Suppression attack."),
    ("Normal.txt", "Everything is normal at the moment."),
    ("VariableLoad.txt", "Everything is normal at the moment."),
    ("UnderFrequency.txt", "There is an Under Frequency disturbance.")
]

# List to store all messages
all_messages = []

# Iterate over each file and its corresponding message
for file_name, message_content in file_messages:
    # Construct the full file path
    file_path = os.path.join(folder_path, file_name)
    
    # Read the content of the text file
    file_content = read_text_file(file_path)
    
    # Construct the message dictionary
    message = {"role": "user", "content": file_content}
    
    # Add the system message before the user message
    system_message = {"role": "system", "content": "You are a anomaly detecting chatbot that looks for cyber-attacks in GOOSE communications at a Smart Grid Power Station."}
    
    # Add the assistant message
    assistant_message = {"role": "assistant", "content": message_content}
    
    # Create the messages dictionary
    messages = [system_message, message, assistant_message]
    
    # Append the messages dictionary to all_messages
    all_messages.append({"messages": messages})

# Write the messages to a JSONL file
output_file_path = "messages.jsonl"
with open(output_file_path, 'w') as output_file:
    for message in all_messages:
        output_file.write(json.dumps(message) + '\n')
